In [1]:
import os
import shutil
import random
from pathlib import Path
import numpy as np
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import GroupKFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pip install ultralytics

In [3]:
from ultralytics import YOLO

In [4]:
model = YOLO("yolov8x-cls.pt")

In [5]:
# make data for the yolo model
root_folder_path = '/kaggle/input/eye-diseases-classification/dataset'
output_folder_path = '/kaggle/working/'

# Create output folders
output_train = os.path.join(output_folder_path, 'dataset/train')
output_val = os.path.join(output_folder_path, 'dataset/validation')
output_test = os.path.join(output_folder_path, 'dataset/test')

os.makedirs(output_train, exist_ok=True)
os.makedirs(output_val, exist_ok=True)
os.makedirs(output_test, exist_ok=True)

In [6]:
output_folder_path = '/kaggle/working/'

In [7]:
# Loop through each class folder
for class_folder in os.listdir(root_folder_path):
    class_path = os.path.join(root_folder_path, class_folder)

    # Skip if not a directory
    if not os.path.isdir(class_path):
        continue

    # Get a list of all image files in the class folder
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # Shuffle the list of images
    random.shuffle(images)

    # Calculate the number of images for each set
    num_images = len(images)
    num_train = int(num_images * 0.80)
    num_val = int(num_images * 0.15)

    # Split the images into train, validation, and test sets
    train_set = images[:num_train]
    val_set = images[num_train:num_train + num_val]
    test_set = images[num_train + num_val:]

    # Move images to their respective folders
    for img in train_set:
        src = os.path.join(class_path, img)
        dest = os.path.join(output_train, class_folder, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

    for img in val_set:
        src = os.path.join(class_path, img)
        dest = os.path.join(output_val, class_folder, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

    for img in test_set:
        src = os.path.join(class_path, img)
        dest = os.path.join(output_test, class_folder, img)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

In [13]:
model.train(data="/kaggle/working/dataset/", epochs=64)

Ultralytics YOLOv8.0.209 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15110MiB)
engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/kaggle/working/dataset/, epochs=64, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opti

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /kaggle/working/dataset/train... 3962 images, 0 corrupt: 100%|██████████| 3962/3962 [00:00<?, ?it/s]
albumentations: RandomResizedCrop(p=1.0, height=224, width=224, scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=1), HorizontalFlip(p=0.5), ColorJitter(p=0.5, brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.30000000000000004, 1.7], hue=[-0.015, 0.015]), Normalize(p=1.0, mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0), max_pixel_value=255.0), ToTensorV2(always_apply=True, p=1.0, transpose_mask=False)
val: Scanning /kaggle/working/dataset/validation... 979 images, 0 corrupt: 100%|██████████| 979/979 [00:00<?, ?it/s]
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=

lr/pg0,▃▆████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▃▆████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▃▆████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/accuracy_top1,▁▃▅▅▅▆▆▆▆▇▇▆▆▇▇▇▆▇▇▇▇▇▇█████████████████
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,█▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,█▇▅▄▅▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg0,3e-05


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a28adabc70>
curves: []
curves_results: []
fitness: 0.9938713312149048
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9877426624298096, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9938713312149048}
save_dir: PosixPath('runs/classify/train22')
speed: {'preprocess': 0.1894589951138209, 'inference': 2.1130580824169125, 'loss': 0.0007106286147276398, 'postprocess': 0.0006947989848587924}
task: 'classify'
top1: 0.9877426624298096
top5: 1.0

In [9]:
results = model('/kaggle/working/dataset/test/glaucoma/_358_8144492.jpg')
#results


image 1/1 /kaggle/working/dataset/test/glaucoma/_358_8144492.jpg: 224x224 glaucoma 1.00, cataract 0.00, normal 0.00, diabetic_retinopathy 0.00, 14.0ms
Speed: 1.2ms preprocess, 14.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


In [12]:
for r in results:
    print(r.probs)

ultralytics.engine.results.Probs object with attributes:

data: tensor([2.0324e-04, 4.1145e-10, 9.9980e-01, 8.9665e-07], device='cuda:0')
orig_shape: None
shape: torch.Size([4])
top1: 2
top1conf: tensor(0.9998, device='cuda:0')
top5: [2, 0, 3, 1]
top5conf: tensor([9.9980e-01, 2.0324e-04, 8.9665e-07, 4.1145e-10], device='cuda:0')
